In [1]:
import pandas as pd
import dash
import dash_html_components as html
import plotly.graph_objs as go
import dash_core_components as dcc
from jupyter_dash import JupyterDash
from dash.dependencies import Input, Output
import json
import plotly.express as px
import dash_cytoscape as cyto

# Load default data

In [2]:

with open('/home/ivana/Environments/Data_Preprocessing/json/data2021.json') as json_file:
    data2021 = json.load(json_file)
    
with open('/home/ivana/Environments/Data_Preprocessing/json/names2021.json') as json_file:
    names2021 = json.load(json_file)


# Generate elements of the graph

In [6]:
def generate_elements(data_dict, names_dict):
    added_pairs = {}
    elements = []
    for cik1 in data_dict.keys():
        issuers_dict1 = data_dict.get(cik1)
        issuers_set1 = set(issuers_dict1.keys())

        for cik2 in data_dict.keys():
            issuers_dict2 = data_dict.get(cik2)
            issuers_set2 = set(issuers_dict2.keys())
            common = issuers_set1.intersection(issuers_set2)

            if (bool(common) and cik1 != cik2):
                e1 = {'data' : {'id': cik1, 'label' : names_dict.get(cik1)}}
                e2 = {'data' : {'id': cik2, 'label' : names_dict.get(cik2)}}
                e3 = {'data': {'source': cik1, 'target': cik2}}
                elements.append(e1)
                elements.append(e2)
                elements.append(e3)
    return elements

# App Layout

In [8]:
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = JupyterDash(__name__, external_stylesheets=external_stylesheets)

# Create server variable with Flask server object for use with gunicorn
server = app.server

# App Layout & Callbacks
app.layout = html.Div([
    
    html.Div(className='two columns', children=[
        dcc.Dropdown(
                id='year-selection',
                options=[
                        {'label': '2018', 'value': '2018'},
                        {'label': '2019', 'value': '2019'},
                        {'label': '2020', 'value': '2020'},
                        {'label': '2021', 'value': '2021'}
                ],
                value='2021'
        ),
    ]),

     html.Div(className='ten columns', children=[
        cyto.Cytoscape(
            id='cytoscape',
            elements= generate_elements(data2021, names2021),
            layout={'name': 'random'},
            #stylesheet=default_stylesheet,
            style={'width': '1000px', 'height': '1000px'}
        )
        ])
])

     

In [9]:
@app.callback(
    Output('cytoscape', 'elements'),
    Input('year-selection', 'value'),
)
def update_nodes(year_value):
    with open('/home/ivana/Environments/Data_Preprocessing/json/data'+str(year_value) + '.json') as json_file:
        data = json.load(json_file)
    with open('/home/ivana/Environments/Data_Preprocessing/json/names'+str(year_value) + '.json') as json_file:
        names = json.load(json_file)
    
    elements = generate_elements(data, names)
    
    return elements


In [10]:
if __name__ == '__main__':
    app.run_server(debug=True)

Dash app running on http://127.0.0.1:8050/
